# 프로젝트 목표: IMDb 영화 데이터를 기반으로 영화의 대중성(popularity_score = Rating × Votes)을 예측하는 머신러닝 모델을 구축

# 1980 - 2025 데이터 합치기

In [213]:
import pandas as pd
import numpy as np

In [214]:
def transform_duration_minutes(df):
    """
    1. 'Duration' 컬럼의 문자열로 되어있는 상영시간을 분석하여,
       'Duration_minute'라는 새 컬럼에 int타입으로 전체 상영시간 분(minute)으로 변환
    2. 'Duration' 컬럼 제거
    3. 결측치(NaN)는 중앙값(median)으로 대체
    """

    # helper function
    def parse_duration(duration):
        if not isinstance(duration, str) or duration.strip() == "":
            return np.nan
        hours, minutes = 0, 0

        if 'h' in duration:
            h_split = duration.split('h')
            hours = int(h_split[0].strip())
            if 'm' in h_split[1]:
                minutes = int(h_split[1].replace('m', '').strip())
        elif 'm' in duration:
            minutes = int(duration.replace('m', '').strip())
        
        return hours * 60 + minutes

    # 1. 변환
    df['Duration_minute'] = df['Duration'].apply(parse_duration)

    # 2. 결측값 -> 중앙값으로 대체
    median_duration = df['Duration_minute'].median()
    df['Duration_minute'] = df['Duration_minute'].fillna(median_duration)

    return df

In [215]:
movie_df = pd.DataFrame(columns=['Title', 'Year', 'Duration', 'MPA', 'Rating', 'Votes', 'méta_score','description', 'Movie Link', 'writers', 'directors', 'stars', 'budget','opening_weekend_Gross', 'grossWorldWWide', 'gross_US_Canada','release_date', 'countries_origin', 'filming_locations','production_company', 'awards_content', 'genres', 'Languages'])

years = list(range(1980, 2026))
for year in years:
    path = f'./movie/movie_data/{year}/merged_movies_data_{year}.csv'
    temp_df = pd.read_csv(path)
    
    # Duration 변환: e.g. 2h 4m -> 124
    temp_df = transform_duration_minutes(temp_df)

    # 기존 dataframe이랑 병합
    movie_df = pd.concat([movie_df,temp_df], axis=0) # 행 기준 병합

movie_df = movie_df.reset_index(drop=True)
# 원래 Duration 컬럼 drop
movie_df = movie_df.drop('Duration', axis=1)

C:\Users\Playdata\AppData\Local\Temp\ipykernel_19308\1185007731.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  movie_df = pd.concat([movie_df,temp_df], axis=0) # 행 기준 병합


In [216]:
# 확인
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27550 entries, 0 to 27549
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Title                  27550 non-null  object 
 1   Year                   27550 non-null  object 
 2   MPA                    21554 non-null  object 
 3   Rating                 27235 non-null  float64
 4   Votes                  27235 non-null  object 
 5   méta_score             14067 non-null  float64
 6   description            27289 non-null  object 
 7   Movie Link             27550 non-null  object 
 8   writers                27550 non-null  object 
 9   directors              27550 non-null  object 
 10  stars                  27323 non-null  object 
 11  budget                 10889 non-null  object 
 12  opening_weekend_Gross  16436 non-null  object 
 13  grossWorldWWide        19029 non-null  object 
 14  gross_US_Canada        17918 non-null  object 
 15  re

## 결측치 행 삭제
- **MPA**: 범주형이고 영화 시나리오에 기반하여 등급이 결정되기 때문에 임의로 값 부여 불가
- **Rating, Votes**: 타겟 임의지표 (popularity_score)를 만들 때 사용하기 때문에 임의 값 부여 불가

In [217]:
movie_df = movie_df.dropna(subset=['MPA', 'Rating', 'Votes'])
movie_df = movie_df.reset_index(drop=True)

## 전혀 필요없는 컬럼 제거
- 후천적 요소 (수입)
- 영화 설명 (시나리오 의미 분석으로 원래는 중요하지만 현재 수준으로 처리 불가)
- 제거 컬럼:
['méta_score', 'description', 'Movie Link', 'opening_weekend_Gross', 'grossWorldWWide', 'gross_US_Canada', 'filming_locations', 'awards_content', 'release_date']

In [218]:
movie_df = movie_df.drop(['méta_score', 'description', 'Movie Link', 'opening_weekend_Gross', 'grossWorldWWide', 'gross_US_Canada', 'filming_locations', 'awards_content', 'release_date'], axis=1)

In [219]:
# 연도 정수 변환
movie_df['Year'] = pd.to_numeric(movie_df['Year'], errors='coerce').astype('Int64')

In [220]:
def parse_abbr_number(text):
    """
    1.3M → 1_300_000
    2.1K → 2_100
    833  → 833
    """
    if not isinstance(text, str):
        return int(text)

    text = text.strip().upper()

    try:
        if text.endswith("M"):
            return int(float(text[:-1]) * 1_000_000)
        elif text.endswith("K"):
            return int(float(text[:-1]) * 1_000)
        elif text.endswith("B"):
            return int(float(text[:-1]) * 1_000_000_000)
        elif text.endswith("T"):
            return int(float(text[:-1]) * 1_000_000_000_000)
        else:
            return int(text.replace(",", ""))
    except:
        return None  # 처리 안되는 경우

In [221]:
movie_df['Votes'] = movie_df['Votes'].apply(parse_abbr_number)

## Budget 숫자 변환 (USD $)로 통일
- 문제: 모든 통화에 대해서 변경을 했는데도 원래부터 반 정도가 결측임

In [222]:
import re

# 환율 (2025년 7월 IMF 기준 또는 실거래 추정 환율)
exchange_rates = {
    'USD': 1.0,
    'EUR': 1.1527,
    'GBP': 1.3364,
    'CNY': 1 / 7.1795,
    'JPY': 1 / 156.23,
    'INR': 1 / 87.2713,
    'KRW': 1 / 1395.71,
    'MXN': 1 / 17.32,
    'AUD': 1 / 1.48,
    'CAD': 1 / 1.37,
    'PLN': 1 / 3.6981,
    'CHF': 1.1364,
    'BRL': 1 / 5.37,
    'SEK': 1 / 10.44,
    'NOK': 1 / 10.59,
    'HKD': 1 / 7.82,
    'NZD': 1 / 1.64,
    'TWD': 1 / 32.4,
    'RUB': 1 / 90.8,
    'SGD': 1 / 1.35,
    'HUF': 1 / 360.2,
    'CZK': 1 / 22.9,
    'ZAR': 1 / 18.2,
    'JMD': 1 / 155.4,
    'ISK': 1 / 140.3,
    'DKK': 1 / 6.88,
    'THB': 1 / 36.0,
    'VEB': 1 / 3600000,
    'CLP': 1 / 910,
    'NGN': 1 / 1570.0,
    'ARS': 1 / 940.0,
    # 유럽 구통화 → 유로 → USD
    'FRF': (1 / 6.55957) * 1.1527,
    'DEM': (1 / 1.95583) * 1.1527,
    'ITL': (1 / 1936.27) * 1.1527,
    'ESP': (1 / 166.386) * 1.1527,
    'NLG': (1 / 2.20371) * 1.1527,
    'FIM': (1 / 5.94573) * 1.1527,
    'BEF': (1 / 40.3399) * 1.1527,
    'IEP': (1 / 0.787564) * 1.1527,
    'ATS': (1 / 13.7603) * 1.1527,
    'TRL': 1 / 1200000  # 예전 터키 리라 (비율 기준치)
}



symbol_amount_patterns = [
    ('USD',  ['$', 'US$', 'USD', 'usd', 'Usd']),
    ('CNY',  ['CN¥', 'CNY', 'cny', 'Cn¥', 'Yuan']),
    ('JPY',  ['¥', 'JPY', 'jpy', 'Yen']),
    ('EUR',  ['€', 'EUR', 'eur', 'Euro']),
    ('GBP',  ['£', 'GBP', 'gbp', 'Pound']),
    ('INR',  ['₹', 'INR', 'inr', 'Rs', 'rs']),
    ('KRW',  ['₩', 'KRW', 'krw', 'Won']),
    ('MXN',  ['MX$', 'MXN', 'mxn']),
    ('AUD',  ['A$', 'AUD', 'aud']),
    ('CAD',  ['CA$', 'CAD', 'cad']),
    ('PLN',  ['PLN', 'pln']),
    ('CHF',  ['CHF', 'chf']),
    ('BRL',  ['R$', 'BRL', 'brl']),
    ('SEK',  ['SEK']),
    ('NOK',  ['NOK']),
    ('HKD',  ['HK$', 'HKD']),
    ('NZD',  ['NZ$', 'NZD']),
    ('TWD',  ['NT$', 'TWD']),
    ('RUB',  ['RUR', 'RUB']),
    ('SGD',  ['SGD']),
    ('HUF',  ['HUF']),
    ('CZK',  ['CZK']),
    ('ZAR',  ['ZAR']),
    ('JMD',  ['JMD']),
    ('ISK',  ['ISK']),
    ('DKK',  ['DKK']),
    ('THB',  ['THB']),
    ('VEB',  ['VEB']),
    ('CLP',  ['CLP']),
    ('NGN',  ['NGN']),
    ('ARS',  ['ARS']),
    # 유럽 구통화
    ('FRF',  ['FRF']),
    ('DEM',  ['DEM']),
    ('ITL',  ['ITL']),
    ('ESP',  ['ESP']),
    ('NLG',  ['NLG']),
    ('FIM',  ['FIM']),
    ('BEF',  ['BEF']),
    ('IEP',  ['IEP']),
    ('ATS',  ['ATS']),
    ('TRL',  ['TRL']),
]


def parse_budget_to_usd(text):
    """
    budget 문자열에서 통화 단위 및 금액을 추출하여
    USD 기준으로 환산된 float 금액을 반환한다.
    """
    if not isinstance(text, str):
        return np.nan

    # (estimated) 같은 괄호 제거 + 쉼표 + 공백 제거
    txt = re.sub(r'\s*\(.*?\)', '', text).strip()
    txt = txt.replace(',', '').replace(' ', '')

    for code, symbols in symbol_amount_patterns:
        for sym in symbols:
            if txt.startswith(sym):
                try:
                    amt_str = txt[len(sym):]
                    amount = float(amt_str)
                    rate = exchange_rates.get(code, np.nan)
                    return round(amount * rate, 2) if not np.isnan(rate) else np.nan
                except:
                    return np.nan
    return np.nan


In [223]:
movie_df['budget_usd'] = movie_df['budget'].apply(parse_budget_to_usd)

In [224]:
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21520 entries, 0 to 21519
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Title               21520 non-null  object 
 1   Year                21520 non-null  Int64  
 2   MPA                 21520 non-null  object 
 3   Rating              21520 non-null  float64
 4   Votes               21520 non-null  int64  
 5   writers             21520 non-null  object 
 6   directors           21520 non-null  object 
 7   stars               21435 non-null  object 
 8   budget              10185 non-null  object 
 9   countries_origin    21511 non-null  object 
 10  production_company  21038 non-null  object 
 11  genres              21520 non-null  object 
 12  Languages           21520 non-null  object 
 13  Duration_minute     21520 non-null  float64
 14  budget_usd          10185 non-null  float64
dtypes: Int64(1), float64(3), int64(1), object(10)
memory 

## 장르 전처리
IMDb에서 정의한 장르만 필터링

In [225]:
import re
import ast

# 1. IMDb 장르 목록
imdb_genres = {
    'Action', 'Adult', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
    'Documentary', 'Drama', 'Family', 'Fantasy', 'Film Noir', 'Game Show',
    'History', 'Horror', 'Musical', 'Music', 'Mystery', 'News', 'Reality-TV',
    'Romance', 'Sci-Fi', 'Short', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western'
}

# 2. 안전하게 리스트로 변환
def to_list_safe(x):
    if isinstance(x, list):
        return x
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return []
    if isinstance(x, str):
        x = x.strip()
        try:
            return ast.literal_eval(x)
        except:
            return [x]
    return [str(x)]

# 3. 장르 문자열 정제
def clean_genre(g):
    #g = g.replace('-', ' ')
    g = re.sub(r'\s+', ' ', g)
    return g.strip().title()

# 4. IMDb 장르 기준 부분 포함 판단
def keep_imdb_only_fuzzy(genre_list):
    cleaned = [clean_genre(g) for g in genre_list]
    matched = set()
    for raw_genre in cleaned:
        for imdb_genre in imdb_genres:
            if imdb_genre.lower() in raw_genre.lower():
                matched.add(imdb_genre)
    return sorted(matched)

# 5. 적용
movie_df['genres'] = movie_df['genres'].apply(to_list_safe)
movie_df['genres_imdb'] = movie_df['genres'].apply(keep_imdb_only_fuzzy)

In [226]:
movie_df = movie_df.reset_index(drop=True)

In [227]:
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21520 entries, 0 to 21519
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Title               21520 non-null  object 
 1   Year                21520 non-null  Int64  
 2   MPA                 21520 non-null  object 
 3   Rating              21520 non-null  float64
 4   Votes               21520 non-null  int64  
 5   writers             21520 non-null  object 
 6   directors           21520 non-null  object 
 7   stars               21435 non-null  object 
 8   budget              10185 non-null  object 
 9   countries_origin    21511 non-null  object 
 10  production_company  21038 non-null  object 
 11  genres              21520 non-null  object 
 12  Languages           21520 non-null  object 
 13  Duration_minute     21520 non-null  float64
 14  budget_usd          10185 non-null  float64
 15  genres_imdb         21520 non-null  object 
dtypes: I

## 리스트처럼 보이지만 문자열로 되어있는 컬럼 리스트로 바꾸기 (이거는 csv로 추출하면 다시 문자열로 저장이 되니 밑에 코드 사용해서 다시 리스트로 변환하면 됩니다)
- '['Richard Lester', 'Richard Donner']' => ['Richard Lester', 'Richard Donner']

In [228]:
import ast

def safe_literal_eval_clean(x):
    # None이나 NaN 값 먼저 걸러냄
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return []

    try:
        # 문자열인 경우: literal_eval 시도
        if isinstance(x, str):
            x = x.replace('""', '"').strip()
            if x.startswith('[') and x.endswith(']'):
                result = ast.literal_eval(x)
            else:
                result = [x]
        # 리스트인 경우: 그대로 사용
        elif isinstance(x, list):
            result = x
        # 그 외는 단일값으로 리스트화
        else:
            result = [x]

        # 결과가 리스트인 경우, 요소를 문자열로 정리
        if isinstance(result, list):
            return [str(item).strip() for item in result if item is not None and str(item).strip() != '']
        else:
            return [str(result).strip()]

    except:
        return []



target_cols = [
    'writers',
    'directors',
    'stars',
    'production_company',
    'Languages',
    'countries_origin',
    'genres_imdb'
]

for col in target_cols:
    movie_df[col] = movie_df[col].apply(safe_literal_eval_clean)


In [229]:
movie_df['genres'].iloc[0]

['Action Epic',
 'Adventure Epic',
 'Dark Fantasy',
 'Epic',
 'Fantasy Epic',
 'Globetrotting Adventure',
 'Quest',
 'Sci-Fi Epic',
 'Space Sci-Fi',
 'Sword & Sorcery']

In [230]:
# 원래 'genre' 컬럼 drop
# 원래 'budget' 컬럼 drop
# 필요없는 're
movie_df = movie_df.drop(['genres','budget'], axis=1)

# 대중성 (= Rating * Votes) 컬럼 추가: 우리가 예측할 타겟

In [231]:
movie_df['popularity_score'] = movie_df['Rating'] * movie_df['Votes']

In [232]:
movie_df = movie_df.reset_index(drop=True)
movie_df

,Title,Year,MPA,Rating,Votes,writers,directors,stars,countries_origin,production_company,Languages,Duration_minute,budget_usd,genres_imdb,popularity_score
0,1. Star Wars: Episode V - The Empire Strikes Back,1980,PG,8.7,1400000,"[Leigh Brackett, Lawrence Kasdan, George Lucas]",[Irvin Kershner],"[Mark Hamill, Harrison Ford, Carrie Fisher, Bi...",[United States],[Lucasfilm],[English],124.0,18000000.0,"[Action, Adventure, Fantasy, Sci-Fi]",12180000.0
1,2. Superman II,1980,PG,6.8,118000,"[Jerry Siegel, Joe Shuster, Mario Puzo]","[Richard Lester, Richard Donner]","[Gene Hackman, Christopher Reeve, Margot Kidde...",[United Kingdom],"[Dovemead Films, Film Export A.G., Internation...","[English, French, Russian]",127.0,54000000.0,"[Action, Adventure, Sci-Fi]",802400.0
2,3. 9 to 5,1980,PG,6.9,41000,"[Patricia Resnick, Colin Higgins]",[Colin Higgins],"[Jane Fonda, Lily Tomlin, Dolly Parton, Dabney...",[United States],"[IPC Films, Twentieth Century Fox]","[English, French]",109.0,10000000.0,[Comedy],282900.0
3,4. Stir Crazy,1980,R,6.7,31000,"[Bruce Jay Friedman, Charles Blackwell]",[Sidney Poitier],"[Gene Wilder, Richard Pryor, Georg Stanford Br...",[United States],[Columbia Pictures],"[English, Spanish]",111.0,10000000.0,"[Comedy, Crime]",207700.0
4,5. Airplane!,1980,PG,7.7,272000,"[Jim Abrahams, David Zucker, Jerry Zucker]","[Jim Abrahams, David Zucker, Jerry Zucker]","[Robert Hays, Julie Hagerty, Leslie Nielsen, K...",[United States],"[Paramount Pictures, Howard W. Koch Productions]",[English],88.0,3500000.0,[Comedy],2094400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21515,434. The Most Beautiful Girl in the World,2025,TV-14,6.1,655,"[Ifan Ismail, Robert Ronny, Titien Wattimena]",[Robert Ronny],"[Reza Rahadian, Sheila Dara Aisha, Kevin Julio...",[Indonesia],[Paragon Pictures],[Indonesian],122.0,NaN,"[Comedy, Drama, Romance]",3995.5
21516,456. Like Father Like Son,2025,R,4.8,46,[Barry Jay],[Barry Jay],"[Dylan Flashner, Dermot Mulroney, Benjamin Mac...",[United States],"[Code 3 Films, Green Light Pictures, Grindston...",[English],89.0,NaN,[Thriller],220.8
21517,493. The Baby in the Basket,2025,TV-MA,5.4,84,"[Tom Jolliffe, Nathan Shepka]","[Andy Crane, Nathan Shepka]","[Maryam d'Abo, Paul Barber, Amber Doig-Thorne,...",[United Kingdom],"[Flickering Myth, Jolliffe Productions, Shepka...",[],100.0,NaN,"[Drama, Horror, Mystery]",453.6
21518,514. Diddy: The Making of a Bad Boy,2025,TV-MA,5.3,539,"[Lisa Bloom, Kim Osorio, Sara Rivers]","[Lisa Bloom, Al B. Sure!]","[Lisa Bloom, Kim Osorio, Sara Rivers, Al B. Su...",[United States],"[AMPLE Entertainment, Blink Films, FGW Product...",[English],99.0,NaN,"[Biography, Crime, Documentary, Music]",2856.7


# CSV로 내보내기

In [233]:
movie_df.to_csv('movies_1980_2025.csv', index=False, quotechar='"', escapechar='\\')